In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv3D, Flatten,MaxPooling3D,AveragePooling3D, concatenate,Input ,SpatialDropout3D,Dropout
import keras
from math import e
from keras.models import Model
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
from keras.models import load_model
from sklearn.decomposition import PCA

In [2]:
modelName=('ACCESS-CM2','EC-Earth3','IITM-ESM','IPSL-CM6A-LR','MPI-ESM1-2-LR')
gridList=('gn','gr','gn','gr','gn')           
#NH_NDJF_zg_day_ACCESS-CM2_ssp370_r1i1p1f1_gn_20150101-21001231.nc
for m,g in zip(modelName,gridList):
    zLevData = xr.open_dataset('../../../Data/ssp370/'+m+'/processed/NH_NDJF_zg_day_'+m+'_ssp370_r1i1p1f1_'+g+'_20150101-21001231.nc')
    zLevData.plev
    z500=zLevData.zg.sel(plev=50000.0,method='nearest')
    z500
    z145E40N=z500.sel(lat=40,lon=145,method='nearest')
    z20E55N=z500.sel(lat=55,lon=20,method='nearest')
    z75E55N=z500.sel(lat=55,lon=75,method='nearest')
    #Calculate Anomaly
    z145E40N_Anom = z145E40N.groupby("time.dayofyear") - z145E40N.groupby("time.dayofyear").mean("time")
    z75E55N_Anom  = z75E55N.groupby("time.dayofyear")  - z75E55N.groupby("time.dayofyear").mean("time")
    z20E55N_Anom  = z20E55N.groupby("time.dayofyear")  - z20E55N.groupby("time.dayofyear").mean("time")
    # calculate std normal anomalies
    z145E40N_AnomStd = z145E40N_Anom.groupby("time.dayofyear")  / z145E40N.groupby("time.dayofyear").std("time")
    z75E55N_AnomStd  = z75E55N_Anom.groupby("time.dayofyear")   / z75E55N.groupby("time.dayofyear").std("time")
    z20E55N_AnomStd  = z20E55N_Anom.groupby("time.dayofyear")   / z20E55N.groupby("time.dayofyear").std("time")


    EUVal = -(1.0/4.0)*z20E55N_AnomStd + (1.0/2.0)*z75E55N_AnomStd - (1.0/4.0)*z145E40N_AnomStd
    EU_ds = xr.Dataset({'EU': (('time'), EUVal)}, coords={'time': zLevData.time})

    EU_ds.to_netcdf(m+'-EUindex-NDJF-Daily-2015-2100.nc')
    
    